<a href="https://colab.research.google.com/github/Samir-atra/FCOS_Implementation/blob/main/model/FCOS_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import libraries

In [ ]:
# imports
# !pip install -U pip
# !pip install -U six numpy wheel packaging
# !pip install -U keras_preprocessing --no-deps
# !pip install opencv-python
# !pip install tf-models-official

# import tensorflow_models as tfm
import tqdm
import tensorflow as tf
import numpy as np
import json
import os
import cv2
import re
from operator import itemgetter


# Download COCO dataset

In [ ]:
# download the dataset 

!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!wget http://images.cocodataset.org/zips/test2014.zip
!wget http://images.cocodataset.org/annotations/image_info_test2014.zip


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Unzip files

In [ ]:
# unzip the dataset files

!pip install unzip
!unzip "/kaggle/working/annotations_trainval2014.zip"
!unzip "/kaggle/working/image_info_test2014.zip"
!unzip "/kaggle/working/test2014.zip"
!unzip "/kaggle/working/train2014.zip"
!unzip "/kaggle/working/val2014.zip"


# Datasets paths

In [ ]:
# dataset paths

# for colab
train_imgs_path = "/content/drive/MyDrive/COCO/train2014/"
annotations = "/content/drive/MyDrive/COCO/annotations/"


#for library
# train_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/train2014/train2014/"
# val_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/val2014/val2014/"
# test_imgs_path = "C:/Users/1845718/Documents/FCOS_Implementation/test2014/test2014/"
# annotations = "C:/Users/1845718/Documents/FCOS_Implementation/annotations_trainval2014/annotations/"

# for laptop
# train_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/train2014/"
# val_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014/"
# test_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/test2014/"
# annotations = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/annotations/"

In [ ]:
# images_dict = dict()
# image_list = []
# counter = 0

# # load the images
# for file in sorted(os.listdir(train_imgs_path)):
#     if counter < 600:
#         image_path = os.path.join(train_imgs_path, file)
#         image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
#         image = cv2.resize(image, (800, 1024))
#         image = np.array(image)
#         image = image.astype('float32')
#         image /= 255
#         if len(image.shape) < 3:
#             continue
#         # plt.imshow(image)
#         # plt.show()
#         file_key = str(file).split('.')[0]
#         imageid = re.findall("[0-9]+", file_key)
#         imageid = str(imageid[1]).lstrip("_0")
#         images_dict[f'{imageid}'] = image           # create a dictionary of the image titles and themselves
#         image_list.append(image)
#         counter += 1
#     else:
#         break

# Util function

In [ ]:
def concat(list1, list2):
    """a concatination function for the COCO dataset bounding box labels"""
    if isinstance(list1[0], list) and isinstance(list2, list):
        alist = list1
        alist.append(list2)
    elif isinstance(list2, int):
        alist = [list1]
        alist.append(list2)
    return alist

# check if need to put all the boxes of a single image in a list instead of keeping them separate.
# concat([1.08, 187.69, 611.59, 285.84],51)

# Analyze the dataset format

In [ ]:
labels_list = []
bounding_boxes = []
counter = 0
dont_care = [0,0,0,0,0,0,0,0,0]
annotation_path = os.path.join(annotations, "instances_train2014.json")
boxes = dict()
counter = 0

annotation_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/annotations/instances_train2014.json"

def calculate_distances():
    """find the distance of the """



def get_regression_target():
    """creates the regression target for the FCOS model"""
    
    boxes = {}
    counter = 0
    
    with open(annotation_path, "r", encoding="utf-8") as f:
        json_obj = json.load(f)
        sorted_imageids = sorted(json_obj["annotations"], key=itemgetter("image_id"))

        for line in sorted_imageids:  
                if counter < 500:                  # due to limited computation had to limit the number of boxes to be loaded
                    if not boxes.get(line["image_id"]):
                        boxes[line["image_id"]] = [concat(line["bbox"], )]
                        
                    elif boxes.get(line["image_id"]):
                        boxes[line["image_id"]] = concat(boxes[line["image_id"]],concat(line["bbox"], line["category_id"]),)
                    
                    counter += 1

    return boxes


# Load the datasets into memory

In [ ]:

class DataLoader:
    """loading the COCO dataset images and bounding boxes as labels,
    contains two methods on for each functionality"""
    def __init__(self, train_images_path, annotations_path):
        self.train_images_path = train_images_path
        self.annotations_path = annotations_path

    def load_images(self, data_path):
        """loads the images of the dataset into a list"""
        images_dict = {}
        image_list = []
        counter = 0
        # load the images
        for file in sorted(tf.io.gfile.listdir(train_imgs_path)):
            if counter < 100:
                image_path = tf.io.gfile.join(train_imgs_path, file)
                image = tf.io.read_file(image_path)
                image = tf.io.decode_image(image, dtype=tf.dtypes.float32)
                image = tf.image.resize(
                    image, (800, 1024), method="bilinear", preserve_aspect_ratio=False
                )
                if len(image.shape) < 3:
                    continue
                # plt.imshow(image)
                # plt.show()
                file_key = str(file).split(".")[0]
                imageid = re.findall("[0-9]+", file_key)
                imageid = str(imageid[1]).lstrip("_0")
                images_dict[f"{imageid}"] = (
                    image  # create a dictionary of the image titles and themselves
                )
                plt.imshow(image)
                plt.show()
                # print(image)
                image_list.append(image)
                counter += 1
            else:
                break
        return image_list


    def load_labels(self, file_path):
        """creates labels lists for each image consisting of the image id and
        category id and the bounding box for each object in the image"""

        labels_list = []
        bounding_boxes = []
        counter = 0
        dont_care = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        annotation_path = tf.io.gfile.join(annotations, "instances_train2014.json")
        boxes = {}
        counter = 0

        with open(annotation_path, "r", encoding="utf-8") as f:
            json_obj = json.load(f)
            sorted_imageids = sorted(
                json_obj["annotations"], key=itemgetter("image_id")
            )
            # draw the bounding boxes
            for (
                line
            ) in (
                sorted_imageids
            ):  # iterate over the lines of the label file, where each line is a box with it's label
                # print(line)
                if counter < 500:                  # due to limited computation had to limit the number of boxes to be loaded
                    if not boxes.get(line["image_id"]):
                        boxes[line["image_id"]] = [
                            concat(line["bbox"], line["category_id"])
                        ]
                    elif boxes.get(line["image_id"]):
                        boxes[line["image_id"]] = concat(
                            boxes[line["image_id"]],
                            concat(line["bbox"], line["category_id"]),
                        )
                    counter += 1

        return boxes


# ResNet backbone model

In [ ]:
# model backbone
# FCOS uses ResNeXt not his one.

def backbone():
    """
    imaplementation of the ResNet backbone model pre-trained 
    on the ImageNet dataset.
    
    Returns:
        c3: the output of the third resnet block
        c4: the output of the forth resnet block
        c5: the output of the fifth resent block
    """
    
    resnet = tf.keras.applications.ResNet50(
            include_top=False,
            weights="imagenet",
            input_shape=(800, 1024, 3),
            classes=80,
        )

    c3, c4, c5 = resnet.get_layer("conv3_block4_out").output, resnet.get_layer("conv4_block6_out").output, resnet.get_layer("conv5_block3_out").output
    
    return c3, c4, c5

exp_config = tfm.core.exp_factory.get_exp_config('resnet_imagenet')
exp_config.task.model.num_classes = 8
exp_config.task.model.input_size = list(1024,800,3)
exp_config.task.model.backbone.resnet.model_id = 50
batch_size = 16


# FPN backbone Model

In [ ]:

class FPN(tf.keras.layers.Layer):
    
    def __init__(self):
        super(FPN, self).__init__()
        self.conv3_1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv4_1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv5_1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv3_3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv4_3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv5_3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv6_3 = tf.keras.layers.Conv2D(256, 3, 2, "same")
        self.conv7_3 = tf.keras.layers.Conv2D(256, 3, 2, "same")
        self.upsample = tf.keras.layers.UpSampling2D(2)

    def call(self, c3, c4, c5):
        """
        the feature pyramid network implementation
            
        Args:
            c3 : the third resnet block output
            c4 : the fourth resnet block output
            c5 : the fifth resnet block outout 

        Returns:
            [p3_out, p4_out, p5_out, p6_out, p7_out] (list): list of the outputs of 
            the third forth fifth sixth seventh layers of the feature pyramid network
        """
        # there are changes to be done here, adding GN
        p3_out = self.conv3_1(c3)
        p4_out = self.conv4_1(c4)
        p5_out = self.conv5_1(c5)
        p4_out = p4_out + self.upsample(p5_out)
        p3_out = p3_out + self.upsample(p4_out)
        p3_out = self.conv3_3(p3_out)
        p4_out = self.conv4_3(p4_out)
        p5_out = self.conv5_3(p5_out)
        p6_out = self.conv6_3(p5_out)
        p7_out = self.conv7_3(tf.nn.relu(p6_out))

        return [p3_out, p4_out, p5_out, p6_out, p7_out]


# Heads function

In [ ]:

def head(output_classes, bias):
    """the output head for the FCOS model from the FPN

    Args:
        output_classes (int): number of classes in the output layer
        bias: the initial bias value
    Output:
        model (keras.model): the head model
    """
    kernel_initial = tf.initializers.RandomNormal(0.0, 0.01)
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=[None, None, 256]),
            tf.keras.layers.Conv2D(
                256, 3, padding="same", kernel_initializer=kernel_initial
            ),
            tf.keras.layers.ReLU(),
            # tf.keras.layers.GroupNormalization(),
            tf.keras.layers.Conv2D(
                256, 3, padding="same", kernel_initializer=kernel_initial
            ),
            tf.keras.layers.ReLU(),
            # tf.keras.layers.GroupNormalization(),
            tf.keras.layers.Conv2D(
                256, 3, padding="same", kernel_initializer=kernel_initial
            ),
            tf.keras.layers.ReLU(),
            # tf.keras.layers.GroupNormalization(),
            tf.keras.layers.Conv2D(
                256, 3, padding="same", kernel_initializer=kernel_initial
            ),
            tf.keras.layers.ReLU(),
            # tf.keras.layers.GroupNormalization(),
            tf.keras.layers.Conv2D(
                output_classes,
                3,
                1,
                padding="same",
                kernel_initializer=kernel_initial,
                bias_initializer=bias,
            ),
        ]
    )

    return model


# Loss functions

In [ ]:

class IOULoss(tf.keras.Loss):
    """Intersection Over Union (IOU) loss function"""

    def __init__(self):
        super().__init__()

    def call(self, pred, g_label):
        """the method includes the implementation of the IOU loss function"""
        if g_label != 0:
            pred_left = pred[0]
            pred_top = pred[1]
            pred_right = pred[2]
            pred_bottom = pred[3]

            g_label_left = g_label[0]
            g_label_top = g_label[1]
            g_label_right = g_label[2]
            g_label_bottom = g_label[3]

            target_area = (g_label_left + g_label_right) * (
                g_label_top + g_label_bottom
            )
            pred_area = (pred_left + pred_right) * (pred_top + pred_bottom)

            w_intersect = tf.math.minimum(pred_left, g_label_left) + tf.math.minimum(
                pred_right, g_label_right
            )
            g_w_intersect = tf.math.maximum(pred_left, g_label_left) + tf.math.maximum(
                pred_right, g_label_right
            )
            h_intersect = tf.math.minimum(
                pred_bottom, g_label_bottom
            ) + tf.math.minimum(pred_top, g_label_top)
            g_h_intersect = tf.math.maximum(
                pred_bottom, g_label_bottom
            ) + tf.math.maximum(pred_top, g_label_top)
            ac_union = g_w_intersect * g_h_intersect + 1e-7
            area_intersect = w_intersect * h_intersect
            area_union = target_area + pred_area - area_intersect

            ious = (area_intersect + 1.0) / (
                area_union + 1.0
            )  # need to check what these are and the next line and compare pytorch to research paper.
            gious = ious - (ac_union - area_union) / ac_union

            loss = -tf.math.log(ious)

            return loss

        else:
            loss = 0
            return loss


class FcosLoss(tf.keras.Loss):
    """implementation of the FCOS loss function"""

    def __init__(self):
        super().__init__()

    def call(self, pred, g_label):
        """
        the method includes the implementation of the FCOS loss function
        Args:
            pred : the prediction of the model
            g_label : the ground truth label of the model
        Returns:
            the loss value
        """
        focal = tf.keras.losses.CategoricalFocalCrossentropy()
        iouloss = IOULoss()
        bce = tf.keras.losses.BinaryCrossentropy()
        
        return focal(pred, g_label) , iouloss(pred, g_label) , bce(pred, g_label)


# Model FCOS

In [ ]:

class FCOS(tf.keras.Model):
    """
    implementation for the FCOS model using the 
    ResNet50 as a backbone
    """
    def __init__(self):
        super(FCOS, self).__init__()
        self.backbone = backbone()
        self.pyramid = FPN()
        self.bias = tf.keras.initializers.Constant(-tf.math.log((1 - 0.01) / 0.01))
        self.class_num = 80
        self.classification_head = head(self.class_num, self.bias)
        self.centerness_head = head(1, self.bias)
        self.box_head = head(4, "zero")
        
    def call(self):
        """
        the full fcos model assembled parts 
        
        Returns:
            list of all the outputs of the model heads 
            for one training example
        """
        c3, c4, c5 = self.backbone()
        p3_out, p4_out, p5_out, p6_out, p7_out = self.pyramid(c3, c4, c5)
        
        classifier_out = []
        box_out = []
        centerness_out = []
        
        for layer in [p3_out, p4_out, p5_out, p6_out, p7_out]:
            classifier_out.append(self.classification_head(layer))
            centerness_out.append(self.centerness_head(layer))
            box_out.append(self.box_head(layer))
        
        classifier_out = tf.concat(classifier_out, axis = 1, name = "classifier")
        centerness_out = tf.concat(centerness_out, axis = 1, name = "centerness")
        box_out = tf.concat(box_out, axis = 1, name = "box")
        
        return tf.concat([classifier_out, centerness_out, box_out], axis = -1)



# loss function
focal = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha = 0.25,
    gamma = 2.0,
)
iouloss = IOULoss()
bce = tf.keras.losses.BinaryCrossentropy()

model = FCOS()

model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01,
              weight_decay = 0.0001,
              momentum = 0.9),
              loss = {'classifier': focal, 'box': iouloss, 'centerness': bce},
              metrics = ['precision'])


def schedule(epoch, lr):
  if epoch == 60000 or epoch == 80000:
    lr = lr / 10
    return lr

sched = tf.keras.callbacks.LearningRateScheduler(
    schedule
)

model.fit(
          epochs = 90000,
          batch_size = 16,
          callbacks = [sched],
          )

